In [1]:
import os

In [2]:
%pwd

'c:\\Users\\HP\\Email_Spam_Prediction\\research'

In [3]:
os.chdir("../")


In [4]:
%pwd


'c:\\Users\\HP\\Email_Spam_Prediction'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    target_column: str

In [6]:
from emailproject.constants import *
from emailproject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.multinomialnb
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            alpha = params.alpha,
            target_column = schema.name
            
        )

        return model_trainer_config

In [8]:
import pandas as pd
import os
from emailproject import logger
from sklearn.naive_bayes import MultinomialNB
import joblib

In [9]:
from sklearn.feature_extraction.text import CountVectorizer


class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config


    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)


        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[self.config.target_column].values
        test_y = test_data[self.config.target_column].values
        
        cv = CountVectorizer()
        train_x = cv.fit_transform(train_x.iloc[:, 0])  # Assuming the text data is in the first column
        test_x = cv.transform(test_x.iloc[:, 0])

        nb = MultinomialNB()
        nb.fit(train_x, train_y.ravel())

        joblib.dump(nb, os.path.join(self.config.root_dir, self.config.model_name))
        

In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-06-09 12:35:23,312: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-06-09 12:35:23,325: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-09 12:35:23,328: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-06-09 12:35:23,328: INFO: common: created directory at: artifacts]
[2024-06-09 12:35:23,331: INFO: common: created directory at: artifacts/model_trainer]
